In [ ]:
import sys

sys.path.insert(0, "../")
sys.path.insert(0, "packages")

In [ ]:
import os

if os.environ.get("CIRCLECI"):
    default_env = os.environ.get("CONDA_DEFAULT_ENV")
    os.environ["PYSPARK_DRIVER_PYTHON"] = (
        f"/home/circleci/miniconda/envs/{default_env}/bin/python"
    )
    os.environ["PYSPARK_PYTHON"] = (
        f"/home/circleci/miniconda/envs/{default_env}/bin/python"
    )

# Utils

This section outlines some of the additional utilities available within the module.
Compared to the rest of the module, these are smaller functions more commonly used
during the data ingestion process.

In [ ]:
from feature_generation.v1.core import utils
import pkgutil


pkgname = utils.__name__
pkgpath = utils.__path__[0]
found_packages = list(pkgutil.iter_modules([pkgpath], prefix="{}.".format(pkgname)))
sub_packages = [x.split(".")[-1] for _, x, _ in found_packages]

importer = found_packages[0][0]

rows = []
for idx, _name in enumerate(sub_packages):
    module_spec = importer.find_spec(found_packages[idx][1])
    module = module_spec.loader.load_module(found_packages[idx][1])
    x_doc = module.__doc__.split("\n")[0]
    rows.append({"sub_module": module.__name__, "description": x_doc})

from tabulate import tabulate
import pandas as pd

table = pd.DataFrame(rows)
print(tabulate(table, headers=table.columns, tablefmt="psql"))

## Summary of Functions

Below is a summary of the available functions within the sub-modules:

In [ ]:
from types import FunctionType


func_row = []
for idx, _name in enumerate(sub_packages):
    module_spec = importer.find_spec(found_packages[idx][1])
    module = module_spec.loader.load_module(found_packages[idx][1])

    list_of_tags = [
        x
        for x in dir(module)
        if isinstance(getattr(module, x), FunctionType) and not x.startswith("_")
    ]

    from tabulate import tabulate

    for x in list_of_tags:
        x_doc = getattr(module, x).__doc__.split("\n")[0]
        func_row.append(
            {"sub_module": module.__name__, "function": x, "description": x_doc}
        )

import pandas as pd

table = pd.DataFrame(func_row)
print(tabulate(table, headers=table.columns, tablefmt="psql"))